## Prodigy

In our patents, a lot of named entities are abbreviated.
For example, `WI-FI Direct` is mentioned as `WFD`, or `P2P Group Owner` as `GO`.

Our original model does not recognize these abbreviations and therefore a huge part of the entities are ignored.
So we will fine-tune our model using prodigy

We create a train dataset for fine-tuning our ner model.

In [1]:
!prodigy ner.correct fine_tune_g06k spacy_output/model-best G06K.txt --loader txt --label TECH

Using 1 label(s): TECH
/Users/gaetanserre/miniconda3/envs/NLP/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

^C

✔ Saved 35 annotations to database SQLite
Dataset: fine_tune_g06k
Session ID: 2022-04-20_12-40-25



We export our dataset.

In [6]:
!prodigy db-out fine_tune_g06k terms_g06k

✔ Exported 35 annotations from 'fine_tune_g06k' in database SQLite
/Users/gaetanserre/Documents/Projects/patent_ner_linking/terms_g06k/fine_tune_g06k.jsonl


Now let's fine-tune our model!

In [2]:
!prodigy train ./prodigy_output/ --ner fine_tune_g06k --base-model spacy_output/model-best

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
ℹ Using config from base model
✔ Generated training config

=========================== Initializing pipeline ===========================
[2022-04-20 12:55:40,635] [INFO] Set up nlp object from config
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 28 | Evaluation: 7 (20% split)
Training: 15 | Evaluation: 5
Labels: ner (1)
[2022-04-20 12:55:40,660] [INFO] Pipeline: ['tok2vec', 'transformer', 'parser', 'ner']
[2022-04-20 12:55:40,660] [INFO] Resuming training for: ['ner', 'transformer']
[2022-04-20 12:55:40,666] [INFO] Created vocabulary
[2022-04-20 12:55:45,445] [INFO] Added vectors: spacy_output/model-best
[2022-04-20 12:55:45,522] [INFO] Finished initializing nlp object
[2022-04-20 12:55:45,707] [INFO] Initialized pipeline components: ['tok2vec']
[2022-04-20 12:55:45,707] [WARNING] [W087] Component 'tok2vec' will be (

In [ ]:
import spacy
nlp = spacy.load("spacy_output/model-best")

In [ ]:
doc = nlp("According to the technology of PTL 1, it is possible to more safely obtain the public key by reading the QR code.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)